# Use scikit-learn and AI lifecycle capabilities to predict Boston house prices with `ibm-watsonx-ai`

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Watson Machine Learning Service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:

-  Download an externally trained scikit-learn model with dataset.
-  Persist an external model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.
-  Update previously persisted model.
-  Redeploy model in-place.
-  Scale deployment.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Download externally created scikit model and data](#download)
3. [Persist externally created scikit model](#persistence)
4. [Deploy and score in a Cloud](#scoring)
5. [Persist new version of the model](#update_model)
6. [Redeploy new version of the model](#redeploy)
7. [Deployment scaling](#scale)
8. [Clean up](#cleanup)
9. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install wget | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install "scikit-learn==1.3.2" | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [1]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.0"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.0"
)

```

In [2]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [4]:
client.set.default_space(space_id)

'SUCCESS'

<a id="download"></a>
## 2. Download externally created scikit model and data
In this section, you will download externally created scikit models and data used for training it.

In [5]:
import os
import wget

data_dir = 'BOSTON_HOUSE_PRICES_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

model_path = os.path.join(data_dir, 'boston_house_prices_model.tar.gz')
new_model_path = os.path.join(data_dir, 'new_boston_house_prices_model.tar.gz')
if not os.path.isfile(model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd5.0/models/scikit/boston_house_price/model/boston_house_prices_model.tar.gz", out=data_dir)  
if not os.path.isfile(new_model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd5.0/models/scikit/boston_house_price/model/new_boston_house_prices_model.tar.gz", out=data_dir)  

In [6]:
from sklearn.datasets import fetch_openml
import pandas as pd

boston = fetch_openml(name='boston', version=1)
boston_df = pd.DataFrame(boston.data)
boston_df.columns = boston.feature_names
boston_df['PRICE'] = boston.target

In [7]:
train_df = boston_df
test_df = boston_df.drop(['PRICE'], axis=1)

<a id="persistence"></a>
## 3. Persist externally created scikit model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the watsonx.ai Client.

### 3.1: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [8]:
sofware_spec_id = client.software_specifications.get_id_by_name("runtime-24.1-py3.11")

In [9]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'External scikit model',
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.3',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sofware_spec_id
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata,
    training_data=train_df)

### 3.2: Get model details

In [10]:
import json

published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "45f12dfe-aa78-5b8d-9f38-0ee223c47309",
      "name": "runtime-24.1-py3.11"
    },
    "type": "scikit-learn_1.3"
  },
  "metadata": {
    "created_at": "2024-05-14T09:36:59.392Z",
    "id": "28b273e6-d855-437e-8ebb-61c37cb8bd8a",
    "modified_at": "2024-05-14T09:37:00.965Z",
    "name": "External scikit model",
    "owner": "1000331001",
    "resource_key": "17c0851d-0d5e-4b46-a5bb-be6b7d8be9b2",
    "space_id": "ddb3354c-1d4b-4bf3-a94f-82038c5979ea"
  },
  "system": {
    "warnings": []
  }
}


### 3.3 Get all models

In [42]:
models_details = client.repository.list_models(limit=10)

<a id="scoring"></a>
## 4. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the watsonx.ai Client.

### 4.1: Create model deployment

#### Create online deployment for published model

In [11]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '28b273e6-d855-437e-8ebb-61c37cb8bd8a' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='53d3f256-db26-48fa-9a12-62bf04c0aa4f'
-----------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.

In [12]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

You can also list existing deployments.

In [ ]:
client.deployments.list(limit=10)

### 4.2: Get deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

### 4.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [13]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [14]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use ``client.deployments.score()`` method to run scoring.

In [15]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [16]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          25.709
        ],
        [
          22.123999999999974
        ]
      ]
    }
  ]
}


<a id="update_model"></a>
## 5. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 5.1: Publish new version of the model

Save the current model version. 

In [ ]:
print(json.dumps(client.repository.create_model_revision(published_model_id), indent=2))

Define new model name and update model content. 

In [17]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'External scikit model - updated'
}

published_model = client.repository.update_model(
    model_uid=published_model_id,
    update_model=new_model_path,
    updated_meta_props=metadata
)

Save new model revision of the updated model. 

In [18]:
new_model_revision = client.repository.create_model_revision(published_model_id)
print(json.dumps(new_model_revision, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "45f12dfe-aa78-5b8d-9f38-0ee223c47309",
      "name": "runtime-24.1-py3.11"
    },
    "type": "scikit-learn_1.3"
  },
  "metadata": {
    "commit_info": {
      "committed_at": "2024-05-14T09:38:47.002Z"
    },
    "created_at": "2024-05-14T09:36:59.392Z",
    "id": "28b273e6-d855-437e-8ebb-61c37cb8bd8a",
    "modified_at": "2024-05-14T09:38:44.186Z",
    "name": "External scikit model - updated",
    "owner": "1000331001",
    "resource_key": "48c8db23-92ab-49d3-b283-000887a2863f",
    "rev": "1",
    "space_id": "ddb3354c-1d4b-4bf3-a94f-82038c5979ea"
  },
  "system": {
    "warnings": []
  }
}


**Note:** Model revisions can be identified by model `id` and `rev` number. 

Get model `rev` number from creation details:

In [19]:
rev_id = new_model_revision['metadata'].get('rev')

You can list existing revisions of the model. 

In [20]:
client.repository.list_models_revisions(published_model_id)

REV                             NAME                   CREATED
0   1  External scikit model - updated  2024-05-14T09:36:59.392Z

### 5.2: Get model details

In [21]:
import json

published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)

print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "45f12dfe-aa78-5b8d-9f38-0ee223c47309",
      "name": "runtime-24.1-py3.11"
    },
    "type": "scikit-learn_1.3"
  },
  "metadata": {
    "commit_info": {
      "committed_at": "2024-05-14T09:38:59.054Z"
    },
    "created_at": "2024-05-14T09:36:59.392Z",
    "id": "28b273e6-d855-437e-8ebb-61c37cb8bd8a",
    "modified_at": "2024-05-14T09:38:47.726Z",
    "name": "External scikit model - updated",
    "owner": "1000331001",
    "resource_key": "48c8db23-92ab-49d3-b283-000887a2863f",
    "space_id": "ddb3354c-1d4b-4bf3-a94f-82038c5979ea"
  },
  "system": {
    "warnings": []
  }
}


<a id="redeploy"></a>
## 6. Redeploy new version of the model

In this section, you'll learn how to redeploy new version of the model by using the watsonx.ai Client.

### 6.1 Redeploy model

In [23]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {
        "id": published_model_id,
        "rev": rev_id
    }
}
    
updated_deployment = client.deployments.update(deployment_id=deployment_id, changes=metadata)

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


Wait for the deployment update: 

In [24]:
import time

status = None
while status not in ['ready', 'failed']:
    print('.', end=' ')
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_id)
    status = deployment_details['entity']['status'].get('state')
    
print("\nDeployment update finished with status: ", status)

. Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

Deployment update finished with status:  ready


### 6.2 Get updated deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

<a id="scale"></a>
## 7. Deployment scaling

In this section, you'll learn how to scale your deployment by creating more copies of stored model with Watson Machine Learning Client.  
This feature is for providing High-Availability and to support higher throughput

### 7.1 Scale deployment

In this example, 2 deployment copies will be made.

In [26]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external scikit model - scaling",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 2
    }
}

In [27]:
scaled_deployment = client.deployments.update(deployment_id, metadata)

Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.


### 7.2 Get scaled deployment details

In [ ]:
print(json.dumps(client.deployments.get_details(deployment_id), indent=2))

### 7.3 Score updated deployment

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [29]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [30]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use client.deployments.score() method to run scoring.

In [31]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [32]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          25.709
        ],
        [
          22.123999999999974
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 8. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd5.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 9. Summary and next steps

You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer  
**Jan Sołtysik**, Intern 

Copyright © 2020-2024 IBM. This notebook and its source code are released under the terms of the MIT License.